### Required Imports

In [1]:
from ragraph.io.csv import from_csv
from ragraph.node import Node
import ragraph
import ragraph.plot
import ragraph.analysis.cluster as cluster
import ragraph.analysis.sequence as seq
from ragraph.analysis import heuristics

## MDM with original clustering

In [2]:
# 1. Load data
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=","
)

# 2. Manual cluster mapping (Node name -> Cluster name)
cluster_mapping = {
    # Process Clusters
    "Receiving client orders": "Receiving client data",
    "Receiving KK layout": "Receiving client data",
    "Arrival of VKs from auction": "check-in flowers",
    "Print WMS sticker": "check-in flowers",
    "Scan WMS sticker": "check-in flowers",
    "Check in LE in VK to WMS system": "check-in flowers",
    "Decide OPBW workload distribution": "Pre-sorting",
    "Print OK/BK sticker": "Pre-sorting",
    "Distribute OK/BK sticker and LE": "Pre-sorting",
    "Move OKs to OPBW": "Pre-sorting",
    "Check-in of OK at OPBW": "Sorting",
    "Move KK from STOR to OPBW queue": "Sorting",
    "Move first KK from queue to OPBW": "Sorting",
    "Receive sorting instructions": "Sorting",
    "Scan LE in OK": "Sorting",
    "Move LE from OK to KK": "Sorting",
    "Scan KK to confirm": "Sorting",
    "Determine KK done at SF-OPBW": "Sorting",
    "Determine KK ready by KK is ready algorithm": "Sorting",
    "Determine KK unready by KK is ready algorithm": "Sorting",
    "Move (un)finished KK to STOR": "Sorting",
    "Check-out of OK at OPBW": "Sorting",
    "Move finished KK to EXCH": "Check-out of flowers",
    "Decouple finished KK": "Check-out of flowers",
    "Arrival of empty KKs": "Check-in of empty KKs",
    "Scan incoming KK to check it in": "Check-in of empty KKs",
    "Move checked-in KKs to STOR": "Check-in of empty KKs",
    "Handling exceptions such as big plants": "Unhappy flow processes",
    "Merging stations when there are less orders": "Unhappy flow processes",
    "Indikken of OKs": "Unhappy flow processes",
    "Adding LEs to a new KK when KK has insufficient space": "Unhappy flow processes",

    # Software Clusters
    "Real-Time Database": "Databases",
    "Historical Database": "Databases",
    "Alarms Database": "Databases",
    "SF-RCVG": "Client Screens",
    "SF-OPBW": "Client Screens",
    "Management Screens": "Client Screens",
    "Mobile Screens": "Mobile Client",
    "WMS/WCS API & BarcodeHandler API": "Mobile Client",
    "Maintenance Service": "Actemium management Services",
    "Actemium Service Manager": "Actemium management Services",
    "Fleet Manager": "AMR Software",
    "Actemium Lowpad Communicator": "AMR Software",
    "Florisoft": "Communication with Florisoft",
    "Host Communicator": "Communication with Florisoft",
    "Host Integration": "Communication with Florisoft",
    "Job Creator": "Task Manager",
    "Job Dispatcher": "Task Manager",
    "Job Watcher": "Task Manager",
    "Customer Allocation Management Algorithm": "Algorithm Calculator",
    "KK is Ready Algorithm": "Algorithm Calculator",

    # Hardware Clusters
    "KK (OutputCart)": "Carts",
    "VK (InputCart)": "Carts",
    "OK (StorageCart)": "Carts",
    "RCVG (Input Positions)": "Receiving Goods Stations (RCVG)",
    "Label Printer (RCVG)": "Receiving Goods Stations (RCVG)",
    "FingerScanner (RCVG)": "Receiving Goods Stations (RCVG)",
    "SF-RCVG Screen": "Receiving Goods Stations (RCVG)",
    "OPBW Sorting Positions": "Sorting Stations (OPBW)",
    "OPBW queue": "Sorting Stations (OPBW)",
    "SF-OPBW Screen": "Sorting Stations (OPBW)",
    "Label Printer (OPBW)": "Sorting Stations (OPBW)",
    "FingerScanner (OPBW)": "Sorting Stations (OPBW)",
}

# 3. Create cluster nodes and assign hierarchy
clusters = {}

for node in g.nodes:
    node_name = str(node.name).strip()

    if node_name in cluster_mapping:
        cluster_name = cluster_mapping[node_name]

        if cluster_name not in clusters:
            cluster_node = Node(
                name=cluster_name,
                kind="cluster"
            )
            g.add_node(cluster_node)
            clusters[cluster_name] = cluster_node

        node.parent = clusters[cluster_name]

# 4. Select only leaf nodes for the MDM
all_leafs = [n for n in g.nodes if n.is_leaf]

# 5. Plot MDM (IMPORTANT: node_kinds includes "cluster")
dsm = ragraph.plot.mdm(
    leafs=all_leafs,
    edges=g.edges,
    node_kinds=[
        "cluster",
        "process",
        "software",
        "hardware",
    ],
    style=ragraph.plot.Style(
        piemap=dict(
            display="labels",
            mode="relative",
            fields=g.edge_labels,
        )
    ),
)

dsm

## MDM with clustering and sequencing algorithms

In [3]:
# Define criticality column (in the Excel file it is called C (calculated))
criticality_col = "criticality"

# 1. Converting csv files into a graph object.
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=","
)

# 2. Define the domains
domain_kinds = ["process", "software", "hardware"]
final_ordered_leafs = []

# 3. Iterate through domains to Cluster and Sequence independently
for kind in domain_kinds:
    target_nodes = g.get_nodes_by_kind(kind)
    target_node_set = set(target_nodes)

    # Create a new empty Graph of the same class as 'g'
    sg = g.__class__()

    if hasattr(sg, "add_nodes"):
        sg.add_nodes(target_nodes)
    else:
        for n in target_nodes:
            sg.add_node(n)
            
    internal_edges = [
        e for e in g.edges 
        if e.source in target_node_set and e.target in target_node_set
    ]
    
    if hasattr(sg, "add_edges"):
        sg.add_edges(internal_edges)
    else:
        for e in internal_edges:
            sg.add_edge(e)
    # ---------------------------------------

    # A. Clustering: Group nodes based on flow within this subgraph
    cluster.hierarchical_markov(sg, weights=criticality_col)
    
    # B. Sequencing: Reorder nodes to minimize feedback loops
    seq.genetic(sg, weights=criticality_col)
    
    # Collect the sorted leaves. 
    final_ordered_leafs.extend(sg.leafs)

# 4. Generate the Plot
dsm = ragraph.plot.mdm(
    leafs=final_ordered_leafs,
    edges=g.edges,
    style=ragraph.plot.Style(
        piemap=dict(
            display="labels",
            mode="relative",
            fields=g.edge_labels,
        ),
    ),
)

dsm

## MDM with Criticality (unclustered)

In [4]:
# Converting csv files into a graph object.
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=",",  # Separates list elements within a cell.
    edge_weights=["criticality"]
)

hardware = [n for n in g.get_nodes_by_kind("hardware")]
software = [n for n in g.get_nodes_by_kind("software")]
process = [n for n in g.get_nodes_by_kind("process")]

dsm = ragraph.plot.mdm(
    leafs = process + software + hardware,
    edges = g.edges,
    style= ragraph.plot.Style(
        piemap=dict(
            display="weights",   
            mode= "relative",
            fields=["criticality"]
        ),
    )
)

# Clustering the graph.
roots = heuristics.markov_gamma(
    graph=g,
    alpha=2,    # Expansion parameter.
    beta=2.0,   # Inflation parameter.
    mu=2.0,     # Evaporation parameter.
    gamma=2.0,   # Bus detection parameter.
    inplace=True
)

dsm

## MDM with criticality (original clusters)

In [5]:
# 1. Load data with criticality weights
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=",",
    edge_weights=["criticality"]  # <--- Added this to load the weights
)

# 2. Manual cluster mapping (Node name -> Cluster name)
cluster_mapping = {
    # Process Clusters
    "Receiving client orders": "Receiving client data",
    "Receiving KK layout": "Receiving client data",
    "Arrival of VKs from auction": "check-in flowers",
    "Print WMS sticker": "check-in flowers",
    "Scan WMS sticker": "check-in flowers",
    "Check in LE in VK to WMS system": "check-in flowers",
    "Decide OPBW workload distribution": "Pre-sorting",
    "Print OK/BK sticker": "Pre-sorting",
    "Distribute OK/BK sticker and LE": "Pre-sorting",
    "Move OKs to OPBW": "Pre-sorting",
    "Check-in of OK at OPBW": "Sorting",
    "Move KK from STOR to OPBW queue": "Sorting",
    "Move first KK from queue to OPBW": "Sorting",
    "Receive sorting instructions": "Sorting",
    "Scan LE in OK": "Sorting",
    "Move LE from OK to KK": "Sorting",
    "Scan KK to confirm": "Sorting",
    "Determine KK done at SF-OPBW": "Sorting",
    "Determine KK ready by KK is ready algorithm": "Sorting",
    "Determine KK unready by KK is ready algorithm": "Sorting",
    "Move (un)finished KK to STOR": "Sorting",
    "Check-out of OK at OPBW": "Sorting",
    "Move finished KK to EXCH": "Check-out of flowers",
    "Decouple finished KK": "Check-out of flowers",
    "Arrival of empty KKs": "Check-in of empty KKs",
    "Scan incoming KK to check it in": "Check-in of empty KKs",
    "Move checked-in KKs to STOR": "Check-in of empty KKs",
    "Handling exceptions such as big plants": "Unhappy flow processes",
    "Merging stations when there are less orders": "Unhappy flow processes",
    "Indikken of OKs": "Unhappy flow processes",
    "Adding LEs to a new KK when KK has insufficient space": "Unhappy flow processes",

    # Software Clusters
    "Real-Time Database": "Databases",
    "Historical Database": "Databases",
    "Alarms Database": "Databases",
    "SF-RCVG": "Client Screens",
    "SF-OPBW": "Client Screens",
    "Management Screens": "Client Screens",
    "Mobile Screens": "Mobile Client",
    "WMS/WCS API & BarcodeHandler API": "Mobile Client",
    "Maintenance Service": "Actemium management Services",
    "Actemium Service Manager": "Actemium management Services",
    "Fleet Manager": "AMR Software",
    "Actemium Lowpad Communicator": "AMR Software",
    "Florisoft": "Communication with Florisoft",
    "Host Communicator": "Communication with Florisoft",
    "Host Integration": "Communication with Florisoft",
    "Job Creator": "Task Manager",
    "Job Dispatcher": "Task Manager",
    "Job Watcher": "Task Manager",
    "Customer Allocation Management Algorithm": "Algorithm Calculator",
    "KK is Ready Algorithm": "Algorithm Calculator",

    # Hardware Clusters
    "KK (OutputCart)": "Carts",
    "VK (InputCart)": "Carts",
    "OK (StorageCart)": "Carts",
    "RCVG (Input Positions)": "Receiving Goods Stations (RCVG)",
    "Label Printer (RCVG)": "Receiving Goods Stations (RCVG)",
    "FingerScanner (RCVG)": "Receiving Goods Stations (RCVG)",
    "SF-RCVG Screen": "Receiving Goods Stations (RCVG)",
    "OPBW Sorting Positions": "Sorting Stations (OPBW)",
    "OPBW queue": "Sorting Stations (OPBW)",
    "SF-OPBW Screen": "Sorting Stations (OPBW)",
    "Label Printer (OPBW)": "Sorting Stations (OPBW)",
    "FingerScanner (OPBW)": "Sorting Stations (OPBW)",
}

# 3. Create cluster nodes and assign hierarchy
clusters = {}

for node in g.nodes:
    node_name = str(node.name).strip()

    if node_name in cluster_mapping:
        cluster_name = cluster_mapping[node_name]

        if cluster_name not in clusters:
            cluster_node = Node(
                name=cluster_name,
                kind="cluster"
            )
            g.add_node(cluster_node)
            clusters[cluster_name] = cluster_node

        node.parent = clusters[cluster_name]

# 4. Select only leaf nodes for the MDM
all_leafs = [n for n in g.nodes if n.is_leaf]

# 5. Plot MDM with criticality weights
dsm = ragraph.plot.mdm(
    leafs=all_leafs,
    edges=g.edges,
    node_kinds=[
        "cluster",
        "process",
        "software",
        "hardware",
    ],
    style=ragraph.plot.Style(
        piemap=dict(
            display="weights",       # <--- Changed to weights
            mode="relative",
            fields=["criticality"],  # <--- Pointing to the loaded weight
        )
    ),
)

dsm

## MDM with criticality (clustering and sequencing algorithms)

In [6]:
# Define criticality column (in the Excel file it is called C (calculated))
criticality_col = "criticality"

# 1. Load data
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=",",
    edge_weights=[criticality_col] 
)

# 2. Define the domains to process independently
domain_kinds = ["process", "software", "hardware"]
final_ordered_leafs = []

# 3. Iterate through domains to Cluster and Sequence independently
for kind in domain_kinds:
    target_nodes = g.get_nodes_by_kind(kind)
    target_node_set = set(target_nodes)

    sg = g.__class__()

    # Add nodes
    if hasattr(sg, "add_nodes"):
        sg.add_nodes(target_nodes)
    else:
        for n in target_nodes:
            sg.add_node(n)
            
    internal_edges = [
        e for e in g.edges 
        if e.source in target_node_set and e.target in target_node_set
    ]
    
    if hasattr(sg, "add_edges"):
        sg.add_edges(internal_edges)
    else:
        for e in internal_edges:
            sg.add_edge(e)
    # -------------------------

    # A. Clustering: Group nodes based on flow, weighted by criticality
    cluster.hierarchical_markov(sg, weights=criticality_col)
    
    # B. Sequencing: Reorder nodes to minimize feedback loops, weighted by criticality
    seq.genetic(sg, weights=criticality_col)
    
    # Collect the sorted leaves from this domain into the master list
    final_ordered_leafs.extend(sg.leafs)

# 4. Generate the Plot
dsm = ragraph.plot.mdm(
    leafs=final_ordered_leafs, # Use the algorithmically sorted leaves
    edges=g.edges,             # Use original global edges
    style=ragraph.plot.Style(
        piemap=dict(
            display="weights",       # Display the weight intensity
            mode="relative",
            fields=[criticality_col] # Visualize the criticality
        ),
    ),
)

dsm